#Libraries installation

In [ ]:
!add-apt-repository -y ppa:jonathonf/ffmpeg-4
!apt update
!apt install -y ffmpeg

In [ ]:
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio
!pip install tensorflow

In [ ]:
! pip install huggingface_hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset, DatasetDict, 

#Download the feature store in Gdrive

In [ ]:
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')
from googleapiclient.http import MediaFileUpload

In [ ]:
# Search for the file with the given name
results = drive_service.files().list(q="name='file_train'").execute()

# Get the first file that matches the search query
file = results['files'][0]

# Print the id of the file
print('File ID: {}'.format(file['id']))

x = drive_service.files().get(fileId=file['id'], fields="*").execute()

In [ ]:
print(x)

In [ ]:
import io
import os
from googleapiclient.http import MediaIoBaseDownload

request = drive_service.files().get_media(fileId=file['id'])

# Use the open() function to create a new file
with open(os.path.join(".", file["name"]), "wb") as fh:
    # Use the io.BytesIO object to hold the file's media data
    media_data = io.BytesIO()

    # Initialize the MediaIoBaseDownload object with the io.BytesIO object
    downloader = MediaIoBaseDownload(media_data, request)

    done = False
    while done is False:
        status, done = downloader.next_chunk()
        print("Download %d%%." % int(status.progress() * 100))

    # Write the file's media data to the file on disk
    fh.write(media_data.getvalue())

In [ ]:
results = drive_service.files().list(q="name='file_test'").execute()

# Get the first file that matches the search query
file = results['files'][0]

request = drive_service.files().get_media(fileId=file['id'])

# Use the open() function to create a new file
with open(os.path.join(".", file["name"]), "wb") as fh:
    # Use the io.BytesIO object to hold the file's media data
    media_data = io.BytesIO()

    # Initialize the MediaIoBaseDownload object with the io.BytesIO object
    downloader = MediaIoBaseDownload(media_data, request)

    done = False
    while done is False:
        status, done = downloader.next_chunk()
        print("Download %d%%." % int(status.progress() * 100))

    # Write the file's media data to the file on disk
    fh.write(media_data.getvalue())

In [ ]:
from datasets import load_from_disk

In [ ]:
common_voice = {}
common_voice["train"] = load_from_disk("file_train")
common_voice["test"] = load_from_disk("file_test")

# Training

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [ ]:
import evaluate

metric = evaluate.load("wer")

In [ ]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

In [ ]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

%cd /content/gdrive/MyDrive/KTH/Scalable

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./improved_whisper_model",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=5000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

In [ ]:
# processor.save_pretrained(training_args.output_dir)

In [ ]:
#trainer.train(resume_from_checkpoint='./improved_whisper_model/checkpoint-4000')
trainer.train()

In [ ]:
kwargs = {
    "dataset_tags": "mozilla-foundation/common_voice_11_0",
    "dataset": "Common Voice 11.0",  # a 'pretty' name for the training dataset
    "dataset_args": "config: nl, split: test",
    "language": "Dutch",
    "model_name": "/improved_whisper",  # a 'pretty' name for our model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
    "tags": "hf-asr-leaderboard",
}

In [ ]:
trainer.push_to_hub(**kwargs)

 # Upload model to Google Drive
 This is redundant because we use huggingface to store the model, but could be usefula as backup.

In [ ]:
for file in os.listdir("./improved_whisper_model"):
    print("UPLOADING", file)
    file_metadata = {"name": "model"}
    media = MediaFileUpload(os.path.join("./improved_whisper_model", file), resumable=True)
    drive_service.files().create(
        body=file_metadata, media_body=media, fields="id"
    ).execute()